<a href="https://colab.research.google.com/github/kamikaze02/COSC-4P96-ASSIGNMENT-1/blob/main/COSC_4P96_ASS1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

def preprocessWithMinMax(image):
  image = tf.cast(image, tf.float32)
  return (image/255.0)

def preprocessWithZScore(image):

  data = tf.cast(image, tf.float32)
  mean= tf.reduce_mean(data)
  std = tf.math.reduce_std(data)

  zScore=(data-mean)/std
  return zScore

def dataAugumentationWithRandomCrop(image):
  data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomCrop(24, 24),        # crop to 24x24
    tf.keras.layers.Resizing(28, 28),          # resize back to 28x28
    tf.keras.layers.RandomTranslation(0.1, 0.1) # translate up to 10% of height/width
])
  image = tf.cast(image, tf.float32)
  image = data_augmentation(image, training=True)

  return image


def dataAugumentationWithRandomNoise(image):
    image = tf.cast(image, tf.float32) # normalize

    noise = tf.random.normal(
        shape=tf.shape(image),
        mean=0.0,
        stddev=0.05
    )

    noisy_image = image + noise

    #noisy_image = tf.clip_by_value(noisy_image, 0.0, 1.0)
    return noisy_image


fashion_mnist = tf.keras.datasets.fashion_mnist
(x,y),_=fashion_mnist.load_data()

#randomize the indexs
indices = np.random.permutation(len(x))

split_index = int(0.8 * len(x))


# training data 80%
trainData=x[indices[:split_index]]
trainLabels=y[indices[:split_index]]


remainingIndices=indices[split_index:]
split_test=int(0.5*len(remainingIndices))
#print(split_test)

#test data 10%
testData=x[indices[:split_test]]
testLabel=y[indices[:split_test]]

#validation
validationData=x[indices[split_test:]]
validationLabel=y[indices[split_test:]]

# Calculate 20% of train data for label
numberLabel=int(0.2*len(trainData))
#print(numberLabel)


x_labeled = trainData[:numberLabel]
y_labeled = trainLabels[:numberLabel]

# 80% of train data for unlabel
x_unlabeled=trainData[numberLabel:]

print(x_unlabeled.shape)

# label dataset
label_dataset = tf.data.Dataset.from_tensor_slices((x_labeled, y_labeled))

#Unlabeled dataset
unlabel_dataset = tf.data.Dataset.from_tensor_slices(x_unlabeled)
#dataset for data processing with minMax
label_dataset1=label_dataset

label_dataset1 =label_dataset1.map(lambda x, y: (preprocessWithMinMax(x), y))
unlabel_dataset1=unlabel_dataset.map(lambda x: (preprocessWithMinMax(x)))


#dataset for data processing with z score
label_dataset2=label_dataset
label_dataset2 =label_dataset2.map(lambda x, y: (preprocessWithZScore(x), y))
unlabel_dataset2=unlabel_dataset.map(lambda x: (preprocessWithZScore(x)))
test_image = tf.reshape(tf.range(1, 17, dtype=tf.float32), (4, 4, 1))

tf.print("Augumented: ", dataAugumentationWithRandomNoise(test_image))




29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
(38400, 28, 28)
Augumented:  [[[1.03505552]
  [2.06085825]
  [2.99126029]
  [3.97349644]]

 [[5.02562284]
  [6.0072484]
  [6.95549059]
  [7.95023346]]

 [[8.99178696]
  [10.0287008]
  [10.9237118]
  [11.9978142]]

 [[12.9670982]
  [14.0339336]
  [15.0236034]
  [15.9756308]]]


In [2]:
import tensorflow as tf
import numpy as np

#load data set
fashion_mnist = tf.keras.datasets.fashion_mnist
(x, y), _ = fashion_mnist.load_data()

#Split to 80% training 10% testing 10% validating
#Randomize indices
#Split training
#Find Remaining indices
#Split the remaining indices into validating and testing data set
indices = np.random.permutation(len(x))

split_train = int(0.8 * len(x))
train_indices = indices[:split_train]
remaining_indices = indices[split_train:]

split_test = int(0.5 * len(remaining_indices))
test_indices = remaining_indices[:split_test]
val_indices = remaining_indices[split_test:]

trainData = x[train_indices]
trainLabels = y[train_indices]

testData = x[test_indices]
testLabels = y[test_indices]

validationData = x[val_indices]
validationLabels = y[val_indices]

# 20% LABEL 80% UNLABELED FROM TRAINING DATA SET

num_labeled = int(0.2 * len(trainData))

x_labeled = trainData[:num_labeled]
y_labeled = trainLabels[:num_labeled]

x_unlabeled = trainData[num_labeled:]


######################################
#DATA PREPARATION PART
######################################

global_mean = np.mean(trainData)
global_std = np.std(trainData)

def preprocess_minmax(image):
    image = tf.cast(image, tf.float32) / 255.0
    image = image[..., tf.newaxis]  # Add channel dimension
    return image
def preprocess_zscore(image):
    image = tf.cast(image, tf.float32)
    image = (image - global_mean) / global_std
    image = image[..., tf.newaxis]
    return image
####################################
#DATA ARGUMENTATION
###################################

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomFlip("vertical")
])

def add_gaussian_noise(image):
    noise = tf.random.normal(
        shape=tf.shape(image),
        mean=0.0,
        stddev=0.05
    )
    image = image + noise
    image = tf.clip_by_value(image, 0.0, 1.0)
    return image

def augment(image):
    image = data_augmentation(image, training=True)
    return image